In [ ]:
import sys; sys.path.insert(0, '../lib')
import jinko_helpers as jinko

# Configuration

# Fill your API key (ex: 'd74cc07e-4a86-4aab-952a-a5814ae6e1ee')
# If not set, it will be retrieved from the JINKO_API_KEY environment variable.
# If environment variable is not set, you will be asked for it interactively
apiKey = None

# Fill your Project Id (ex: '14495063-918c-441a-a898-3131d70b02b0')
# If not set, it will be retrieved from the JINKO_PROJECT_ID environment variable.
# If environment variable is not set, you will be asked for it interactively
projectId = None

# This function ensures that authentication is correct
# It it also possible to override the base url by passing baseUrl=...
jinko.initialize(projectId, apiKey = apiKey)

In [ ]:
import zipfile
import io
from typing import List, Any

# Fill the short Id of your Trial (ex: tr-EKRx-3HRt)
trialId = 'tr-GgOU-4DTm'
if trialId is None:
  raise Exception('Please specify a Trial Id')

# Convert short Id to coreItemId
coreItemId = jinko.getCoreItemId(trialId, 1)

# List all Trial versions
versions: List[Any] = jinko.makeRequest('/core/v2/trial_manager/trial/%s/status'%(coreItemId['id'])).json()

# Get the latest completed version
latestCompletedVersion = next((item for item in versions if item['status'] == 'completed'), None)
if latestCompletedVersion is None:
  raise Exception('No completed Trial version found')

# Retrieve scalar results
response = jinko.makeRequest('/core/v2/result_manager/scalars_summary', method='POST', json={
  "select": [
    "x-max-on-medium-minus-its-control",
  ],
  "trialId": latestCompletedVersion['simulationId']
})
archive = zipfile.ZipFile(io.BytesIO(response.content))
filename = archive.namelist()[0]
csv = archive.read(filename).decode('utf-8')

print(csv)